In [7]:
### 1. Mount google drive ####
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [8]:
### 2. prepare data ###
!scp '/content/gdrive/My Drive/Colab Notebooks/Text_Detection/data.zip' '/content/data.zip'

!unzip '/content/data.zip' -d '/content/'

Archive:  /content/data.zip
replace /content/Train/BARTLETT_DENTAL_STATION.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: /content/Train/BARTLETT_DENTAL_STATION.jpg  
replace /content/Train/BOOK_RETURN.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: /content/Train/BOOK_RETURN.jpg  
replace /content/Train/CAFE_&_GALLERY.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: /content/Train/CAFE_&_GALLERY.jpg  
replace /content/Train/Caffe_Artigiano_Coffee_House_&_Bistro.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: /content/Train/Caffe_Artigiano_Coffee_House_&_Bistro.jpg  
replace /content/Train/cANNERY_ROW_BREWING_COMPANY.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: /content/Train/cANNERY_ROW_BREWING_COMPANY.jpg  
replace /content/Train/Chateau_Fujisawa.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: /content/Train/Chateau_Fujisawa.jpg  
replace /content/Train/CHill_CHill_by_Boat_Noodle.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename:

In [9]:

### 3. Install dependencies ###

!apt install tesseract-ocr
!apt install libtesseract-dev

!pip install pytesseract
!pip install Pillow
!pip install easyocr
!pip install boto3

y
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libtesseract-dev is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [10]:
### 4. text detection###
import pytesseract
from PIL import  Image
from easyocr import Reader
import boto3

reader = Reader(['en'])

#///////////
# this is we can try it from IAM AWS , that we create a
access_key = None
secret_access_key = None

textract_client = boto3.client('textract',
                               aws_access_key_id=access_key,
                               aws_secret_access_key = secret_access_key,
                               region_name='us-east-1')

#/////////////

def read_text_tesseract(image_path) :
  text = pytesseract.image_to_string(Image.open(image_path) , lang='eng')
  return text

def read_text_easyocr(image_path):
  text =''
  results =reader.readtext(Image.open(image_path))
  for result in results :
    text += result[1] + ' '
  text = text[:-1]
  return text


def read_text_textract(image_path):
  with open(image_path , 'rb') as im:
    response = textract_client.detect_document_text(Document={'Bytes':im.read()})

    text =''
    for item in response['Blocks']:
      if item['BlockType'] == 'LINE':
        text += item['Text']+ ' '
    text = text[-1]
    return text




In [11]:
### 5. compare performances ####

import os

def jaccard_similarity(sentence1, sentence2):
    # Tokenize sentences into sets of words
    set1 = set(sentence1.lower().split())
    set2 = set(sentence2.lower().split())

    # Calculate Jaccard similarity
    intersection_size = len(set1.intersection(set2))
    union_size = len(set1.union(set2))

    # Avoid division by zero if both sets are empty
    similarity = intersection_size / union_size if union_size != 0 else 0.0

    return similarity

# all of rest commented parts of code, because of necessaty of access key by for example IAM AWS :
score_tesseract = 0
score_easyocr = 0
#score_textract = 0
data_path = '/content/Train'
num_images = len(os.listdir(data_path))

for image_path_ in os.listdir(data_path):
  image_path = os.path.join(data_path, image_path_)

  gt = image_path_[:-4].replace('_', ' ').lower()

  score_tesseract += jaccard_similarity(gt, read_text_tesseract(image_path).lower().replace('\n', '').replace('!', '').replace('?', '').replace('.', ''))
  score_easyocr += jaccard_similarity(gt, read_text_easyocr(image_path).lower().replace('\n', '').replace('\n', '').replace('!', '').replace('?', '').replace('.', ''))
  #score_textract += jaccard_similarity(gt, read_text_textract(image_path).lower().replace('\n', '').replace('!', '').replace('?', '').replace('.', ''))

print('score tesseract:', score_tesseract / num_images)
print('score_easyocr:', score_easyocr / num_images)
#we would have a better result by the final methode (scoretextract)
#print('score_textract:', score_textract / 100)

score tesseract: 0.0
score_easyocr: 0.20628399522780377
